# Data Preparation

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import os

from dask import dataframe as dd

## Import Data

In [2]:
# Set data path for Kaggle
if os.environ.get('KAGGLE_KERNEL_RUN_TYPE', 'Localhost') == 'Interactive':
    loan_data_backup = pd.read_csv("/kaggle/input/loan-data-2007-2014/loan_data_2007_2014/loan_data_2007_2014.csv")
else:
    loan_data_backup = pd.read_csv('../data/loan_data_2007_2014.csv')

/tmp/ipykernel_4483/3054711046.py:5: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  loan_data_backup = pd.read_csv('../data/loan_data_2007_2014.csv')


In [3]:
loan_data = loan_data_backup.copy()

## Explore Data

In [4]:
# loan_data

In [5]:
pd.options.display.max_columns = None

In [6]:
# loan_data #.compute()

In [7]:
# loan_data.head()

In [8]:
# loan_data.tail()

In [9]:
# loan_data.columns.values

In [10]:
# loan_data.info()

## General Preprocessing

### Preprocessing few continuous variables

In [11]:
loan_data['emp_length'].unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', nan],
      dtype=object)

In [12]:
loan_data['emp_length_int'] = loan_data['emp_length'].str.replace('+ years', '')
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace('< 1 year', str(0))
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace('n/a',  str(0))
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace(' years', '')
loan_data['emp_length_int'] = loan_data['emp_length_int'].str.replace(' year', '')

In [13]:
loan_data["emp_length_int"].head()

0    10
1     0
2    10
3    10
4     1
Name: emp_length_int, dtype: object

In [14]:
type(loan_data['emp_length_int'][0])

str

In [15]:
loan_data['emp_length_int'] = pd.to_numeric(loan_data['emp_length_int'])

# loan_data["emp_length_int"] = dd.to_numeric(loan_data['emp_length_int'])

In [16]:
type(loan_data['emp_length_int'][0])

numpy.float64

In [17]:
# loan_data['earliest_cr_line']

In [18]:
loan_data['earliest_cr_line_date'] = pd.to_datetime(loan_data['earliest_cr_line'], format = '%b-%y')

In [19]:
type(loan_data['earliest_cr_line_date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [20]:
# (pd.to_datetime('2017-12-01') - loan_data['earliest_cr_line_date']) #.compute()

In [21]:
def mnth_diff(x):
    end_date = pd.to_datetime('2017-12-01')
    year_diff = end_date.year - x.year 
    mnth_diff = end_date.month - x.month
    return year_diff*12 + mnth_diff + 1

In [22]:
# loan_data['mths_since_earliest_cr_line'] = round(pd.to_numeric((pd.to_datetime('2017-12-01') - loan_data['earliest_cr_line_date']) / np.timedelta64(1, 'M')))
loan_data['days_since_earliest_cr_line'] = pd.to_datetime('2017-12-01') - loan_data['earliest_cr_line_date']
loan_data['mths_since_earliest_cr_line'] = (pd.to_datetime('2017-12-01').to_period("M") - loan_data['earliest_cr_line_date'].dt.to_period("M")).apply(lambda x: x.n if pd.notnull(x) else x).fillna(np.nan).astype('Int64')
# loan_data["mths_since_earliest_cr_line_1"] = loan_data["earliest_cr_line_date"].apply(mnth_diff)
# loan_data['mths_since_earliest_cr_line'] = (pd.to_datetime('2017-12-01').to_period("M") - loan_data['earliest_cr_line_date'].dt.to_period("M")) #.apply(lambda x: x.n if pd.notnull(x) else x).fillna(np.nan).astype('Int64')
loan_data.loc[:, ["days_since_earliest_cr_line", "mths_since_earliest_cr_line"]] #, "mths_since_earliest_cr_line_1"]]

/tmp/ipykernel_4483/2951109446.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  loan_data['mths_since_earliest_cr_line'] = (pd.to_datetime('2017-12-01').to_period("M") - loan_data['earliest_cr_line_date'].dt.to_period("M")).apply(lambda x: x.n if pd.notnull(x) else x).fillna(np.nan).astype('Int64')


,days_since_earliest_cr_line,mths_since_earliest_cr_line
0,12022 days,395
1,6819 days,224
2,5874 days,193
3,7974 days,262
4,8005 days,263
...,...,...
466280,5358 days,176
466281,7488 days,246
466282,5844 days,192
466283,5417 days,178


In [23]:
loan_data['mths_since_earliest_cr_line'].describe()

count     466256.0
mean     239.48243
std      93.974829
min         -612.0
25%          183.0
50%          225.0
75%          285.0
max          587.0
Name: mths_since_earliest_cr_line, dtype: Float64

In [24]:
loan_data.loc[: , ['earliest_cr_line', 'earliest_cr_line_date', 'mths_since_earliest_cr_line']][loan_data['mths_since_earliest_cr_line'] < 0]

,earliest_cr_line,earliest_cr_line_date,mths_since_earliest_cr_line
1580,Sep-62,2062-09-01,-537
1770,Sep-68,2068-09-01,-609
2799,Sep-64,2064-09-01,-561
3282,Sep-67,2067-09-01,-597
3359,Feb-65,2065-02-01,-566
...,...,...,...
464003,Jan-68,2068-01-01,-601
464260,Jul-66,2066-07-01,-583
465100,Oct-67,2067-10-01,-598
465500,Sep-67,2067-09-01,-597


In [25]:
loan_data["mths_since_earliest_cr_line"].max()

587

In [26]:
loan_data.loc[(loan_data['mths_since_earliest_cr_line'] < 0), "mths_since_earliest_cr_line"] = loan_data['mths_since_earliest_cr_line'].max()

In [27]:
# loan_data['mths_since_earliest_cr_line'].min()

In [28]:
loan_data["term"].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [29]:
loan_data["term_int"] = loan_data["term"].str.strip().str.replace(" months", "").astype(int)

In [30]:
loan_data['term_int']

0         36
1         60
2         36
3         36
4         60
          ..
466280    60
466281    60
466282    60
466283    36
466284    36
Name: term_int, Length: 466285, dtype: int64

In [31]:
loan_data["issue_dt"] = pd.to_datetime(loan_data["issue_d"], format = "%b-%y")

In [32]:
loan_data["issue_dt"]

0        2011-12-01
1        2011-12-01
2        2011-12-01
3        2011-12-01
4        2011-12-01
            ...    
466280   2014-01-01
466281   2014-01-01
466282   2014-01-01
466283   2014-01-01
466284   2014-01-01
Name: issue_dt, Length: 466285, dtype: datetime64[ns]

In [33]:
loan_data["mths_since_issue_d"] = (pd.to_datetime('2017-12-01').to_period("M") - loan_data['issue_dt'].dt.to_period("M")).apply(lambda x: x.n if pd.notnull(x) else x).fillna(np.nan).astype('Int64')

In [34]:
loan_data.loc[:, ["mths_since_issue_d", "issue_d"]]

,mths_since_issue_d,issue_d
0,72,Dec-11
1,72,Dec-11
2,72,Dec-11
3,72,Dec-11
4,72,Dec-11
...,...,...
466280,47,Jan-14
466281,47,Jan-14
466282,47,Jan-14
466283,47,Jan-14


In [35]:
loan_data["emp_length"]

0         10+ years
1          < 1 year
2         10+ years
3         10+ years
4            1 year
            ...    
466280      4 years
466281    10+ years
466282      7 years
466283      3 years
466284    10+ years
Name: emp_length, Length: 466285, dtype: object

### Preprocessing few discrete variables

In [36]:
# loan_data.info()

In [37]:
pd.get_dummies(loan_data['grade'])

,A,B,C,D,E,F,G
0,False,True,False,False,False,False,False
1,False,False,True,False,False,False,False
2,False,False,True,False,False,False,False
3,False,False,True,False,False,False,False
4,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...
466280,False,False,True,False,False,False,False
466281,False,False,False,True,False,False,False
466282,False,False,False,True,False,False,False
466283,True,False,False,False,False,False,False


In [38]:
pd.get_dummies(loan_data['grade'], prefix = 'grade', prefix_sep = ':')

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,grade:G
0,False,True,False,False,False,False,False
1,False,False,True,False,False,False,False
2,False,False,True,False,False,False,False
3,False,False,True,False,False,False,False
4,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...
466280,False,False,True,False,False,False,False
466281,False,False,False,True,False,False,False
466282,False,False,False,True,False,False,False
466283,True,False,False,False,False,False,False


In [39]:
# Add drop_first = True, to get dummy with k-1 columns

loan_data_dummies = [pd.get_dummies(loan_data['grade'], prefix = 'grade', prefix_sep = ':', drop_first=True),
                     pd.get_dummies(loan_data['sub_grade'], prefix = 'sub_grade', prefix_sep = ':', drop_first=True),
                     pd.get_dummies(loan_data['home_ownership'], prefix = 'home_ownership', prefix_sep = ':', drop_first=True),
                     pd.get_dummies(loan_data['verification_status'], prefix = 'verification_status', prefix_sep = ':', drop_first=True),
                     pd.get_dummies(loan_data['loan_status'], prefix = 'loan_status', prefix_sep = ':', drop_first=True),
                     pd.get_dummies(loan_data['purpose'], prefix = 'purpose', prefix_sep = ':', drop_first=True),
                     pd.get_dummies(loan_data['addr_state'], prefix = 'addr_state', prefix_sep = ':', drop_first=True),
                     pd.get_dummies(loan_data['initial_list_status'], prefix = 'initial_list_status', prefix_sep = ':', drop_first=True)]

In [40]:
loan_data_dummies = pd.concat(loan_data_dummies, axis = 1)

In [41]:
type(loan_data_dummies)

pandas.core.frame.DataFrame

In [42]:
loan_data = pd.concat([loan_data, loan_data_dummies], axis = 1)

In [43]:
# loan_data.columns.values

## Check null

In [44]:
pd.options.display.max_rows = None

In [45]:
# loan_data.isnull().sum()

In [46]:
loan_data.fillna({'total_rev_hi_lim': loan_data['funded_amnt']}, inplace = True)

In [47]:
loan_data['total_rev_hi_lim'].isnull().sum()

0

In [48]:
loan_data["annual_inc"] = loan_data["annual_inc"].mean()

In [49]:
# pd.options.display.max_rows = None
# loan_data.columns.values

In [50]:
loan_data.fillna({'mths_since_earliest_cr_line':0,
                  'acc_now_delinq':0,
                  'total_acc':0,
                  'pub_rec':0,
                  'open_acc':0,
                  'inq_last_6mths':0,
                  'delinq_2yrs':0,
                  'emp_length_int':0}, inplace=True)

# PD model

## Data preparation

### Dependent Variable. Good/ Bad (Default) Definition. Default and Non-default Accounts.

In [51]:
loan_data['loan_status'].unique()

array(['Fully Paid', 'Charged Off', 'Current', 'Default',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [52]:
loan_data['loan_status'].value_counts()

loan_status
Current                                                224226
Fully Paid                                             184739
Charged Off                                             42475
Late (31-120 days)                                       6900
In Grace Period                                          3146
Does not meet the credit policy. Status:Fully Paid       1988
Late (16-30 days)                                        1218
Default                                                   832
Does not meet the credit policy. Status:Charged Off       761
Name: count, dtype: int64

In [53]:
loan_data['loan_status'].value_counts() / loan_data['loan_status'].count()

loan_status
Current                                                0.480878
Fully Paid                                             0.396193
Charged Off                                            0.091092
Late (31-120 days)                                     0.014798
In Grace Period                                        0.006747
Does not meet the credit policy. Status:Fully Paid     0.004263
Late (16-30 days)                                      0.002612
Default                                                0.001784
Does not meet the credit policy. Status:Charged Off    0.001632
Name: count, dtype: float64

In [54]:
loan_data['good_bad'] = np.where(loan_data['loan_status'].isin(['Charged Off', 'Default',
                                                               'Does not meet the credit policy. Status:Charged Off',
                                                               'Late (31-120 days)']), 0, 1)

## Splitting data

In [55]:
from sklearn.model_selection import train_test_split

In [56]:
loan_data_inputs_train, loan_data_inputs_test, loan_data_targets_train, loan_data_targets_test = train_test_split(loan_data.drop(columns=["good_bad"]), loan_data["good_bad"], test_size=0.2, random_state=42)

In [57]:
loan_data_inputs_train.shape

(373028, 200)

In [58]:
loan_data_inputs_test.shape

(93257, 200)

In [59]:
loan_data_targets_train.shape

(373028,)

In [60]:
loan_data_targets_test.shape

(93257,)

In [61]:
df_inputs_prepr = loan_data_inputs_train
df_target_prepr = loan_data_targets_train

In [62]:
df_inputs_prepr["grade"].unique()

array(['A', 'C', 'D', 'B', 'E', 'F', 'G'], dtype=object)

In [63]:
df1 = pd.concat([df_inputs_prepr["grade"], df_target_prepr], axis = "columns")
df1.head()

,grade,good_bad
427211,A,1
206088,C,1
136020,A,1
412305,D,0
36159,C,0


In [64]:
gr_cnt = df1.groupby(["grade", "good_bad"], as_index=False).agg(cnt = ("grade", "count"))

In [65]:
tt_cnt = df1.groupby(["good_bad"], as_index=False).agg(tt_cnt = ("grade", "count"))

In [66]:
tt_cnt

,good_bad,tt_cnt
0,0,40778
1,1,332250


In [67]:
pv = (gr_cnt
 .merge(tt_cnt,on="good_bad",how="left")
 .assign(pct = lambda x : x["cnt"]/x["tt_cnt"])
 .replace({"good_bad": {0: "bad", 1:"good"}})
 .pivot(index="grade", columns="good_bad", values="pct")
 .assign(woe = lambda x : np.log(x["good"]/x["bad"]))
)

In [68]:
pv

good_bad,bad,good,woe
grade,,,
A,0.057090,0.172855,1.107830
B,0.212541,0.304178,0.358476
C,0.280813,0.267251,-0.049503
D,0.231792,0.156647,-0.391843
E,0.136642,0.069345,-0.678267
F,0.063417,0.023910,-0.975440
G,0.017706,0.005815,-1.113459


In [72]:
inf_val = pv.assign(inf_val = lambda x : (x["good"] - x["bad"]) * x["woe"])["inf_val"].sum()

In [73]:
inf_val

0.2886356129021057